In [2]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
import sklearn.datasets
wine = sklearn.datasets.load_wine()
wine.data.shape

(178, 13)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:, :4], 
    wine.target, 
    test_size=0.3, 
    shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [4]:
class WineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(WineNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(4, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        #self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        #self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        #x = self.fc2(x)
        #x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
wine_net = WineNet(100)

In [5]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(wine_net.parameters(), 
                             lr=1.0e-3)

In [6]:
np.random.permutation(5)

array([2, 0, 1, 4, 3])

In [7]:
batch_size = 10

for epoch in range(5000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = wine_net.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor(0.4074)
tensor(0.7407)
tensor(0.7778)
tensor(0.8148)
tensor(0.8148)


KeyboardInterrupt: ignored

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 8)

n_classes = 5
plot_colors = ['g', 'orange', 'black', 'y', 'b']
plot_step = 0.001

x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
z_min, z_max = X_train[:, 2].min() - 1, X_train[:, 2].max() + 1
t_min, t_max = X_train[:, 3].min() - 1, X_train[:, 3].max() + 1



xx, yy =  torch.meshgrid(torch.arange(x_min, x_max, plot_step),
                         torch.arange(y_min, y_max, plot_step))
zz, tt =  torch.meshgrid(torch.arange(z_min, z_max, plot_step),
                    torch.arange(y_min, y_max, plot_step))
#size 38679681, 2

final = torch.cat([xx.reshape(-1, 1)[:26104221, :], yy.reshape(-1, 1)[:26104221, :]], dim = 1)
final2 = torch.cat([zz.reshape(-1, 1), tt.reshape(-1, 1)], dim = 1)

preds1 = wine_net.inference(final)

# preds_class = preds.data.numpy().argmax(axis=1)
# preds_class = preds_class.reshape(xx.shape)
# plt.contourf(xx, yy, preds_class, cmap='Accent')

# for i, color in zip(range(n_classes), plot_colors):
#     indexes = np.where(y_train == i)
#     plt.scatter(X_train[indexes, 0], 
#                 X_train[indexes, 1], 
#                 c=color, 
#                 label=wine.target_names[i],
#                 cmap='Accent')
#     plt.xlabel(wine.feature_names[0])
#     plt.ylabel(wine.feature_names[1])
#     plt.legend()

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: ignored